## AGENTS from scratch using Tool Calling / binding LLMS (NEW TRENDS)

(Code for Computer Vision Africa - Workshop)

Author: Tariq Jamil

(TensorDot Solutions - www.tensordotsolutions.com)

Linkedin: https://www.linkedin.com/in/tj-yazman/

Dated: 26. October, 2024

In [10]:
from IPython.display import Image, display
import re
import json

from Tools_r3 import calculate, currency_converter, get_news, ddg_search, get_weather
from Tools_r3 import get_tool_specifications, cprint
from jinja2 import Template

#### Defining LLM

In [11]:
import os
import openai
from typing import List, Union, Optional, Dict, Any

class ChatClient:
    def __init__(
        self,
        api_key: Optional[str] = None,
        base_url: str = "https://api.groq.com/openai/v1",
        model: str = "llama-3.1-70b-versatile",
        temperature: float = 0.0,
        max_tokens: int = 512,
        stream: bool = False,
    ):
        """
        Initializing the chat client with default settings.
        """
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        self.base_url = base_url
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.stream = stream
        self.tools = []  #  tools as an empty list in the begining

        # Initializing OpenAI client
        self.client = openai.OpenAI(
            base_url=self.base_url,
            api_key=self.api_key,
        )

    def bind_tools(self, tools: List[Dict[str, Any]]):
        """
        Binding tools to the chat client.
        """
        self.tools = tools

    def run(self, message: Union[str, List[Dict[str, Any]]]) -> Dict[str, Any]:
        """
        Running the chat client with specified message(s), returning the assistant's response.
        """
        # This is to facilitate single query input, for testing etc.
        if isinstance(message, str):
            messages = [
                {"role": "system", "content": "you are a helpful assistant."},
                {"role": "user", "content": message}
            ]
        else:
            messages = message

        # Preparing the API call parameters
        params = {
            "messages": messages,
            "model": self.model,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
            "stream": self.stream,
        }

        if self.tools:  # Including tools only if bind_tool use to add tools
            params["tools"] = self.tools
            params["tool_choice"] = 'auto'  # Set tool_choice as needed
            
        # Calling client to generate response
        chat_completion = self.client.chat.completions.create(**params)

        # Returning assistant's response
        return chat_completion.choices[0].message


### The system prompt can be broken down into 3 portions for understanfing sake.
1. System prompt -- persona, system_message (ReAct methodology)
2. Tools Ingestion (Tools description, and parameters they take)

#### System prompt

In [12]:
name = "Assistant"
role = "A general purpose assistant capable of answering user questions"

system_prompt_template = """
You are an intelligent '{{ name }}' and you act as '{{ role }}':
"""

#### For Agents we need to maintain a state of messages from begining till last until answer is evaluated.

The Chat Models has a dictionary style prompt, have 'role' keys and 'content' keys.

### STATE & Other Initializations

In [13]:
system_prompt = Template(system_prompt_template).render(name=name, role=role)  # tools not specified
# initializations
max_iterations = 20

# Initializing state
messages_state = [{"role": "system", "content": system_prompt}]

#user_query = "What are top 5 news from capital of Nigeria. Also sum the numbers in '245323'"
user_query = "What is the current temperature (celius) at capital of Nigeria."
user_query1 = "What is rain condition Lagos, with time and date."
user_query2 = "What is capital of Nigeria and 3 major cities?, in bullets form"
user_query3 = "what is todays top 4 news items in Nigeria, in bullets form"
user_query4 = "What are 3 best supermarkets in Abuja, in bullets form"

### Tools Schemas

https://platform.openai.com/docs/guides/function-calling

In [14]:
# Import the schemas
from tool_schemas import (
    tool_schema_get_news,
    tool_schema_ddg_search,
    tool_schema_get_weather,
    tool_schema_calculate
)

# Now you can use the schemas in main.py
#print(tool_schema_get_news)
tools_spec = [
    tool_schema_get_news,
    tool_schema_ddg_search,
    tool_schema_get_weather,
    tool_schema_calculate] 

available_functions = {'get_news': get_news, 'ddg_search': ddg_search, 'get_weather': get_weather, "calculate": calculate}

In [15]:
user_query3 = "what is todays top 4 news items in Nigeria, in bullets form"
user_query = 'What is the current temperature (Celsius) at the capital of Nigeria, what is 2*3/80.'
user_query3 = 'weather in Abuja, give buleted response, with current time and date'


In [16]:
import json
client = ChatClient()
client.bind_tools(tools_spec) # registering tools with the client

# Now using the function in your existing workflow
messages_state.append({"role": "user", "content": user_query3})
# Making the initial request
response_message = client.run(messages_state)

tool_calls = response_message.tool_calls
#print(tool_calls)
# Processing tool calls
messages_state.append(response_message)

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]  # Ensure available_functions is defined
    function_args = json.loads(tool_call.function.arguments)
    
    # Calling the function with arguments
    function_response = function_to_call(**function_args)
    #print(function_response)

    messages_state.append(
        {
            "role": "tool",
            "content": str(function_response),
            "tool_call_id": tool_call.id,
        }
    )

# Making the final request with tool call results
final_response = client.run(messages_state)
print(final_response.content)

 -> get_weather Tool Called --

*
***
* Current Time: 16:52
* Current Date: 2024-10-26
* Location: Abuja, Nigeria
* Temperature: 30.7°C (87.3°F)
* Condition: Patchy rain nearby
* Wind Speed: 5.1 mph (8.3 km/h)
* Humidity: 57%
* Cloud Cover: 74%
* Feels Like: 34.0°C (93.2°F)
* Dew Point: 21.3°C (70.4°F)
* Air Quality: Moderate (US-EPA Index: 3, GB-Defra Index: 6)


In [17]:
import json
class Agent:
    def __init__(self, system_prompt: str, llm_client: ChatClient, available_functions: dict):
        self.client = llm_client
        self.available_functions = available_functions  # Available functions for tool calls
        self.messages_state = [{"role": "system", "content": system_prompt}]  # Initialize with system prompt

    def run(self, user_query: str):
        # Appending the user query to the message state
        self.messages_state.append({"role": "user", "content": user_query})

        # Making the initial request
        response_message = self.client.run(self.messages_state)
        print(response_message)
        tool_calls = response_message.tool_calls

        # Appending the response message to the message state
        self.messages_state.append(response_message)

        if tool_calls:
            # Processing tool calls
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = self.available_functions[function_name]  # Ensure available_functions is defined
                function_args = json.loads(tool_call.function.arguments)

                # Calling the function with arguments
                function_response = function_to_call(**function_args)

                # Updating the message state with the tool call results
                self.messages_state.append(
                    {
                        "role": "tool",
                        "content": str(function_response),
                        "tool_call_id": tool_call.id,
                    }
                )
        else:
            print("No tool calls found.")
        # Making the final request with tool call results
        final_response = self.client.run(self.messages_state)
        return final_response.content


In [18]:
llama3 = ChatClient()
llama3.bind_tools(tools_spec)

myAgent = Agent(system_prompt=system_prompt, llm_client=llama3, available_functions=available_functions)
response = myAgent.run('Weather in Abuja')

ChatCompletionMessage(content='<function=get_weather>{"location": "Abuja"}<function>', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
No tool calls found.
